In [58]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')

load_dotenv(find_dotenv())
sys.path.append(os.environ.get('PROJECT_ROOT'))

from src.utils import IdealDataInterface
from src.utils import IdealMetadataInterface

ideal_path = Path(os.environ.get('PROJECT_IDEAL'))
data_path = Path(os.environ.get('PROJECT_DATA'))

folder_path = ideal_path / 'household_sensors' / 'sensordata'
ideal = IdealDataInterface(folder_path)

out_path = data_path / 'raw_gas_data'

Found entries for 1592 sensor readings.


In [55]:
all_homes = ideal.sensorid_mapping.index.get_level_values('homeid')
all_homes = all_homes.unique().tolist()

total_demands = pd.Series(np.zeros(len(all_homes)), index=all_homes)

for home in all_homes:
    gas = ideal.get(homeid=home, subtype='gas')[0]['readings'] 
    
    gas = gas.resample('h').sum()
    gas = gas.divide(1000.)
    
    gas.to_csv(out_path / f'home_{home}.csv') 

In [61]:
mdi = IdealMetadataInterface(ideal_path / 'metadata')

metadata = mdi._metafile('home')['home']
metadata.to_csv(out_path / 'metadata.csv')